In [ ]:
import math
import pandas as pd
import numpy as np
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt
%matplotlib inline



# Reading data

In [ ]:
traindata_df = pd.read_csv('data/TrainData2.csv', delimiter = ',').dropna().reset_index(drop=True)
traindata_df['TIMESTAMP'] = pd.to_datetime(traindata_df['TIMESTAMP'], format='%Y%m%d %H:%M')
traindata_df = traindata_df.set_index('TIMESTAMP')
std_dev = 3
traindata_df = traindata_df[(np.abs(stats.zscore(traindata_df)) < float(std_dev)).all(axis=1)]

In [ ]:
forecast_df = pd.read_csv('data/WeatherForecastInput2.csv', delimiter = ',')
forecast_df['TIMESTAMP'] = pd.to_datetime(forecast_df['TIMESTAMP'], format='%Y%m%d %H:%M')
forecast_df = forecast_df.set_index('TIMESTAMP')

In [4]:
results_st2 = pd.read_csv('data/Solution2.csv', delimiter = ',')
results_st2['TIMESTAMP'] = pd.to_datetime(results_st2['TIMESTAMP'], format='%Y%m%d %H:%M')
results_st2 = results_st2.set_index('TIMESTAMP')

# Feature Engineering

In [5]:
X = pd.DataFrame()
X['U10'] = traindata_df['U10']
X['U100'] = traindata_df['U100']
X['V10'] = traindata_df['V10']
X['V100'] = traindata_df['V100']
X['U10^2'] = X['U10']**2
X['U100^2'] = X['U100']**2
X['V10^2'] = X['V10']**2
X['V100^2'] = X['V100']**2
X['U10^3'] = X['U10']**3
X['U100^3'] = X['U100']**3
X['V10^3'] = X['V10']**3
X['V100^3'] = X['V100']**3

X['W10'] = (np.sqrt(traindata_df['U10']**2 + traindata_df['V10']**2))
X['W100'] = (np.sqrt(traindata_df['U100']**2 + traindata_df['V100']**2))
X['W10^2'] = X['W10']**2
X['W100^2'] = X['W100']**2
X['W10^3'] = X['W10']**3
X['W100^3'] = X['W100']**3
X['cosTheta10'] = traindata_df['U10']/X['W10']
X['sinTheta10'] = traindata_df['V10']/X['W10']
X['cosTheta100'] = traindata_df['U100']/X['W100']
X['sinTheta100'] = traindata_df['V100']/X['W100']

X['beta_0'] = 1

X['hour'] = X.index.hour
X['year'] = X.index.year
X['month'] = X.index.month

hour_df = pd.get_dummies(X['hour'])
X = X.drop(['hour'], axis=1)
X = X.join(hour_df, how = 'left')

y = traindata_df[['POWER']].copy()

X_pred = pd.DataFrame()

X_pred['U10'] = forecast_df['U10']
X_pred['U100'] = forecast_df['U100']
X_pred['V10'] = forecast_df['V10']
X_pred['V100'] = forecast_df['V100']
X_pred['U10^2'] = X_pred['U10']**2
X_pred['U100^2'] = X_pred['U100']**2
X_pred['V10^2'] = X_pred['V10']**2
X_pred['V100^2'] = X_pred['V100']**2
X_pred['U10^3'] = X_pred['U10']**3
X_pred['U100^3'] = X_pred['U100']**3
X_pred['V10^3'] = X_pred['V10']**3
X_pred['V100^3'] = X_pred['V100']**3

X_pred['W10'] = (np.sqrt(forecast_df['U10']**2 + forecast_df['V10']**2))
X_pred['W100'] = (np.sqrt(forecast_df['U100']**2 + forecast_df['V100']**2))
X_pred['W10^2'] = X_pred['W10']**2
X_pred['W100^2'] = X_pred['W100']**2
X_pred['W10^3'] = X_pred['W10']**3
X_pred['W100^3'] = X_pred['W100']**3
X_pred['cosTheta10'] = forecast_df['U10']/X_pred['W10']
X_pred['sinTheta10'] = forecast_df['V10']/X_pred['W10']
X_pred['cosTheta100'] = forecast_df['U100']/X_pred['W100']
X_pred['sinTheta100'] = forecast_df['V100']/X_pred['W100']

X_pred['beta_0'] = 1

X_pred['hour'] = X_pred.index.hour
X_pred['year'] = X_pred.index.year
X_pred['month'] = X_pred.index.month

hour_pred_df = pd.get_dummies(X_pred['hour'],prefix='h')
X_pred = X_pred.drop(['hour'], axis=1)
X_pred = X_pred.join(hour_pred_df, how = 'left')


# Third stage

In [13]:
#X1 = sm.add_constant(X_pred)
#X_pred = sm.add_constant(X_pred)
model = sm.arima_model(y,X)

AttributeError: module 'statsmodels.api' has no attribute 'arima_model'

In [7]:
results = model.fit()

In [8]:
pred = results.predict(X_pred)
sqrt(mean_squared_error(results_st2,pred))

0.1503318994237806